# Logging

Like all problems in data science, performance monitoring starts with collecting the right data in the right format. Because performance monitoring is a concern in nearly all customer-facing computer systems, there is a well-established set of tools and techniques for collecting this data. Data performance monitoting is generally collected using log files. Recall the following best practice:

**Important**
* Ensure that your data are collected at the most granular level possible. This means each data point should represent one user making one action or one event.

Naturally, collecting very granular data will result in very large data sets. If there is a need, you can always summarize the data after it has been collected. Summary level data may mask important patterns and generally it is not possible to go from summary data to granular data. Log files with several million entries can be analyzed on a single node or on a personal machine with little overhead.

**Note: If the logging is handled by another member of your team or by another team you should ensure that the minimally required data discussed here are available or it will be difficult to monitor your model's performance and/or debug performance related issues.**

# Minimal requirements for log files

These are data that are minimally required for performance monitoring for most model deployment projects. There are other features that fall into this category that are situation dependent, like user_id in a recommendation system, so do not view this list as comprehensive, simply keep it as a reference starting point.

**runtime** - The total amount of time required to process the request. This is a factor that directly affects the end user’s experience and should be monitored.

**timestamp** - Timestamps are needed to evaluate how well the system handles load and concurrency. Additionally, timestamps are useful when connecting predictions to labels that are acquired afterwards. Finally, they are needed for the investigation of events that might affect the relationship between the performance and business metrics.

**prediction** - The prediction is, of course, the primary output of a prediction model. It is necessary to track the prediction for comparison to feedback to determine the quality of the predictions. Generally, predictions are returned as a list to accommodate multi-label classification.

**input_data_summary** - Summarizing information about the input data itself. For the predict endpoint this is the shape of the input feature matrix, but for the training endpoint the features and targets should be summarized.

**model_version_number** - The model version number is used to better understand the influence of model improvements (or bugs) on performance.

### Additional features that can be optionally logged

These are the features that are considered nice to have, but they are not always relevant to the circumstances or sometimes there can be practical limitations (e.g. disk space or computational resources) that limit the ability to save these features.

**request_unique_id** - Each request that has been made should correspond to an entry in the log file. It is possible that a request corresponds to more than one entry in the log file for example if more than one model is called. This is also known as correlation_id.

**data** - Saving the input features that were provided at the time of a predict request makes it much easier to debug broken requests. Saving the features and target at the time of training makes it easier to debug broken model training.

**request_type** - Relevant attributes about the request (e.g. webapp request, browser request)

**probability** - Probability associated with a prediction (if applicable)


The value of logging most of the mentioned data is fairly intuitive, but saving the data itself might seem unnecessary. If we save the input features, when a predict endpoint was hit, we can reconstruct the individual prediction, stepping through each part of the prediction process. For training, the archiving of all the data is often unnecessary, because there is a system in place, like a centralized database, that can re-create the training data for a given point in time. One option is to archive only the previous iteration of training data.

# Logging in python

Python has a logging module which can be used for performance monitoring, but we will show logging through the use of the csv module to keep the process as simple as possible. The following code shows how to create a log file.

In [1]:
#!pip install joblib

In [2]:
%%writefile example-logging.py

#!/usr/bin/env python
"""
use the iris data to demonstrate how logging is tied to 
a machine learning model to enable performance monitoring
"""

import time,os,re,csv,sys,uuid,joblib
from datetime import date
import numpy as np
from sklearn import svm
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


def train_model(X,y,saved_model):
    """
    function to train model
    """

    ## Perform a train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    ## Specify parameters and model
    params = {'C':1.0,'kernel':'linear','gamma':0.5}
    clf = svm.SVC(**params,probability=True)

    ## fit model on training data
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(classification_report(y_test,y_pred))

    ## retrain using all data
    clf.fit(X, y)
    print("... saving model: {}".format(saved_model))
    joblib.dump(clf,saved_model)

    
def _update_predict_log(y_pred,y_proba,query,runtime):
    """
    update predict log file
    """

    ## name the logfile using something that cycles with date (day, month, year)    
    today = date.today()
    logfile = "example-predict-{}-{}.log".format(today.year, today.month)

    ## write the data to a csv file    
    header = ['unique_id','timestamp','y_pred','y_proba','x_shape','model_version','runtime']
    write_header = False
    if not os.path.exists(logfile):
        write_header = True
    with open(logfile,'a') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quotechar='|')
        if write_header:
            writer.writerow(header)

        to_write = map(str,[uuid.uuid4(),time.time(),y_pred,y_proba,query.shape,MODEL_VERSION,runtime])
        writer.writerow(to_write)

def predict(query):
    """
    generic function for prediction
    """

    ## start timer for runtime
    time_start = time.time()
    
    ## ensure the model is loaded
    model = joblib.load(saved_model)

    ## output checking
    if len(query.shape) == 1:
        query = query.reshape(1, -1)
    
    ## make prediction and gather data for log entry
    y_pred = model.predict(query)
    y_proba = None
    if 'predict_proba' in dir(model) and model.probability == True:
        y_proba = model.predict_proba(query)
    m, s = divmod(time.time()-time_start, 60)
    h, m = divmod(m, 60)
    runtime = "%03d:%02d:%02d"%(h, m, s)

    ## update the log file
    _update_predict_log(y_pred,y_proba,query,runtime)
    
    return(y_pred)

if __name__ == "__main__":

    ## import some data to play with
    iris = datasets.load_iris()
    X = iris.data[:,:2]
    y = iris.target

    ## train the model
    MODEL_VERSION = 1.0
    saved_model = "example-predict-{}.joblib".format(re.sub("\.","_",str(MODEL_VERSION)))
    model = train_model(X,y,saved_model)

    ## example predict
    for query in [np.array([[6.1,2.8]]), np.array([[7.7,2.5]]), np.array([[5.8,3.8]])]:
        y_pred = predict(query)
        print("predicted: {}".format(y_pred))


Overwriting example-logging.py


In [3]:
!python example-logging.py

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       0.67      0.53      0.59        15
           2       0.63      0.75      0.69        16

   micro avg       0.78      0.78      0.78        50
   macro avg       0.77      0.76      0.76        50
weighted avg       0.78      0.78      0.78        50

... saving model: example-predict-1_0.joblib
predicted: [1]
predicted: [2]
predicted: [0]


In [4]:
logs = open("example-predict-2020-4.log", "r").read()
print(logs)

unique_id,timestamp,y_pred,y_proba,x_shape,model_version,runtime
38f9d8a9-f508-4377-aaa5-d0731b806f0d,1588078438.7247682,[1],[[0.00655896 0.55274448 0.44069655]],|(1, 2)|,1.0,000:00:00
9e655f8d-8173-4a2b-802e-ea5b7c9cde74,1588078438.7315822,[2],[[3.78813625e-05 1.70410037e-01 8.29552082e-01]],|(1, 2)|,1.0,000:00:00
047d2ea0-843c-4ec5-b637-78d057993c25,1588078438.7370567,[0],[[0.71067191 0.13553971 0.15378837]],|(1, 2)|,1.0,000:00:00
df31294f-a806-45b1-81db-191c39630598,1588078805.3078685,[1],[[0.00605679 0.58266977 0.41127344]],|(1, 2)|,1.0,000:00:00
af437e22-6fb9-4c99-ba68-38a4d0188dc6,1588078805.3134978,[2],[[2.10800331e-05 1.31788410e-01 8.68190510e-01]],|(1, 2)|,1.0,000:00:00
cdb88097-6fe9-4532-b2c9-f51d341b0d36,1588078805.3194745,[0],[[0.72022465 0.13827636 0.14149899]],|(1, 2)|,1.0,000:00:00



It is reasonable to use JSON or a centralized database as a target destination. There are numerous other tools like Elasticsearch and Apache Commons Logging. We use simple CSV formatted files because they help keep the Docker container isolated from other environments and they are a convenient format for most data scientists.